In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Interactive RCL temperature and water level data


To run, click Cell -> Run All
--

[<img src="./PyViz_logo_wm.png" width="42" align="left">](http://pyviz.org)





In [3]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh', logo=False)

In [4]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [5]:
#temperature data
names = ['dt', 'temp', 'err']

dft = pd.read_csv('https://fits.geonet.org.nz/observation?siteID=RU001&typeID=t', skiprows=1, names=names, parse_dates=['dt'], index_col='dt')
#resample as daily mean and interpolate
dfdt = dft.resample('D').mean().interpolate(method='linear')

In [6]:
#level data
names = ['dt', 'dep', 'err']

dfz = pd.read_csv('https://fits.geonet.org.nz/observation?siteID=RU001A&typeID=z', skiprows=1, names=names, parse_dates=['dt'], index_col='dt')
#resample as daily mean and interpolate
dfdz = dfz.resample('D').mean().interpolate(method='linear')

In [7]:
#holoviews requires a non-datetime index
dftday = dfdt.reset_index()
dfzday = dfdz.reset_index()

In [8]:
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[
    ("Date", "@dt{%Y-%m-%d}")
], formatters={"dt":"datetime"}, mode='vline')
# hover = HoverTool(tooltips=[
#     ("Date", "@dt{%Y-%m-%d}"),
#     ("Temperature", "@temp"),
#     ("Depth", "@dep")
# ], formatters={"dt":"datetime"}, mode='vline')

In [9]:
%%opts Curve  [height=300 width=1200 tools=[hover]]

#horizontal lines for descrptive statistics
# median = hv.HLine(dftday['temp'].median()).options(color='green', line_width=2, line_dash='dashed', alpha=0.5)
# q25 = hv.HLine(dftday['temp'].quantile(q=0.25)).options(color='black', line_width=2, line_dash='dashed', alpha=0.5)
# q75 = hv.HLine(dftday['temp'].quantile(q=0.75)).options(color='red', line_width=2, line_dash='dashed', alpha=0.5)

#label horizontal lines
# median_label = hv.Text(pd.to_datetime('1951-01-01'), dftday['temp'].median()*1.025, "median", halign='left').options(color='green', text_alpha=0.5)
# q25_label = hv.Text(pd.to_datetime('1951-01-01'), dftday['temp'].quantile(q=0.25)*1.025, "lower quartile", halign='left').options(color='black', text_alpha=0.5)
# q75_label = hv.Text(pd.to_datetime('1951-01-01'), dftday['temp'].quantile(q=0.75)*1.025, "upper quartile", halign='left').options(color='red', text_alpha=0.5)

# curve = hv.Curve(dftday, 'dt', 'temp').redim.label(dt=' ',temp='Temperature (C)') * median * q25 * q75  * median_label * q25_label * q75_label
curve = (hv.Curve(dftday, 'dt', 'temp').redim.label(dt=' ',temp='Temperature (C)').relabel('Ruapehu Crater lake daily temperature') + hv.Curve(dfzday, 'dt', 'dep').redim.label(dt=' ',dep='Depth (m)').relabel('Ruapehu Crater lake daily depth')).cols(1)
curve

:Layout
   .Curve.Ruapehu_Crater_lake_daily_temperature :Curve   [dt]   (temp)
   .Curve.Ruapehu_Crater_lake_daily_depth       :Curve   [dt]   (dep)